<a href="https://colab.research.google.com/github/Wachacha-jay/Computer_vision_Image_classification/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install torchinfo

In [3]:
import os
import sys
from collections import Counter

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from torch.utils.data import DataLoader, random_split
from torchinfo import summary
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

torch.backends.cudnn.deterministic = True

In [4]:
print("Platform:", sys.platform)
print("Python version:", sys.version)
print("---")
print("matplotlib version:", matplotlib.__version__)
print("pandas version:", pd.__version__)
print("PIL version:", PIL.__version__)
print("torch version:", torch.__version__)
print("torchvision version:", torchvision.__version__)

Platform: linux
Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
---
matplotlib version: 3.10.0
pandas version: 2.2.2
PIL version: 11.2.1
torch version: 2.6.0+cu124
torchvision version: 0.21.0+cu124


In [5]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Using {device} device.")

Using cpu device.


In [6]:
!pip install kagglehub

# Upload your Kaggle API key (kaggle.json)
from google.colab import files
files.upload()  # Select your kaggle.json file

# Move the key to the correct directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [7]:
import kagglehub

# Download the dataset
path = kagglehub.dataset_download("srg9000/cassava-plant-disease-merged-20192020")

print("Path to dataset files:", path)  # This will show where the dataset is stored

100%|██████████| 10.7G/10.7G [02:10<00:00, 88.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/srg9000/cassava-plant-disease-merged-20192020/versions/3


In [8]:
import shutil
import os

# Copy the dataset to a new location
destination = "./cassava_dataset"
if not os.path.exists(destination):
    shutil.copytree(path, destination)
    print(f"Dataset copied to {destination}")
else:
    print(f"{destination} already exists")

Dataset copied to ./cassava_dataset


In [9]:
destination

'./cassava_dataset'

In [10]:
os.listdir(destination)

['extraimages_tfrecords',
 'extra_images',
 'merged_train.csv',
 'train_images_tfrecords',
 'train_images']

In [11]:
data_dir = os.path.join(destination, 'merged_train.csv')

In [12]:
import pandas as pd
df = pd.read_csv(data_dir)
df.head()

,image_id,labels
0,723977410.jpg,3
1,3193577634.jpg,1
2,3349107107.jpg,3
3,2933959901.jpg,0
4,3020460837.jpg,4


In [13]:
labels_dict = {
    0: 'Cassava Bacterial Blight (CBB)',
    1: 'Cassava Brown Streak Disease (CBSD)',
    2: 'Cassava Green Mottle (CGM)',
    3: 'Cassava Mosaic Disease (CMD)',
    4: 'Healthy'
}

# Add a new column 'disease_name' by mapping 'labels' to 'labels_dict'
df['disease_name'] = df['labels'].map(labels_dict)

# Display the updated DataFrame
df.head()

,image_id,labels,disease_name
0,723977410.jpg,3,Cassava Mosaic Disease (CMD)
1,3193577634.jpg,1,Cassava Brown Streak Disease (CBSD)
2,3349107107.jpg,3,Cassava Mosaic Disease (CMD)
3,2933959901.jpg,0,Cassava Bacterial Blight (CBB)
4,3020460837.jpg,4,Healthy


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27053 entries, 0 to 27052
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   image_id      27053 non-null  object
 1   labels        27053 non-null  int64 
 2   disease_name  27053 non-null  object
dtypes: int64(1), object(2)
memory usage: 634.2+ KB


In [15]:
train_path = os.path.join(destination, "train_images", "train_images")

In [16]:
import os
import shutil
import pandas as pd

# ===== CONFIGURATION =====
source_folder = train_path  # Folder containing all your images
destination_root = "train"        # Main folder to be created


# ===== FOLDER CREATION =====
# Create main folder and subfolders (with existence checks)
os.makedirs(destination_root, exist_ok=True)  # Main 'train' folder
print(f"✅ Created main folder: {destination_root}/")

for disease in labels_dict.values():
    subfolder = os.path.join(destination_root, disease)
    os.makedirs(subfolder, exist_ok=True)
    print(f"  ├── Created subfolder: {disease}/")

# ===== IMAGE ORGANIZATION =====
success_count = 0
missing_count = 0

for _, row in df.iterrows():
    src = os.path.join(source_folder, row['image_id'])
    dest = os.path.join(destination_root, labels_dict[row['labels']], row['image_id'])

    try:
        shutil.copy2(src, dest)  # copy2 preserves metadata
        success_count += 1
    except FileNotFoundError:
        print(f"⚠️ Missing: {row['image_id']}")
        missing_count += 1
    except Exception as e:
        print(f"❌ Error copying {row['image_id']}: {str(e)}")

# ===== RESULTS SUMMARY =====
print("\n📊 Organization Complete!")
print(f"✅ Successfully copied: {success_count} files")
print(f"⚠️  Missing files: {missing_count}")
print(f"📁 Final structure in: {os.path.abspath(destination_root)}/")

✅ Created main folder: train/
  ├── Created subfolder: Cassava Bacterial Blight (CBB)/
  ├── Created subfolder: Cassava Brown Streak Disease (CBSD)/
  ├── Created subfolder: Cassava Green Mottle (CGM)/
  ├── Created subfolder: Cassava Mosaic Disease (CMD)/
  ├── Created subfolder: Healthy/

📊 Organization Complete!
✅ Successfully copied: 27053 files
⚠️  Missing files: 0
📁 Final structure in: /content/train/


In [17]:
classes = os.listdir("/content/train")

In [18]:
classes

['Healthy',
 'Cassava Green Mottle (CGM)',
 'Cassava Mosaic Disease (CMD)',
 'Cassava Brown Streak Disease (CBSD)',
 'Cassava Bacterial Blight (CBB)']

In [19]:
destination_root = os.path.join("/content/train")

In [20]:
from pathlib import Path
destination_root = Path(destination_root)

In [21]:
type(destination_root)

pathlib.PosixPath

In [22]:
train_dir = list(destination_root.iterdir())
train_dir[0]

PosixPath('/content/train/Healthy')

In [26]:
healthy_dir =train_dir[0]
sample_img_path = list(healthy_dir.iterdir())[0]
sample_img_path

PosixPath('/content/train/Healthy/train-healthy-159.jpg')

In [29]:
#lets investigate one of the images in the healthy directory
sample_img = PIL.Image.open(sample_img_path)
sample_img
print(f"The image has a shape of: {sample_img.size}.")
print(f"The image has a mode of: {sample_img.mode}.")

The image has a shape of: (888, 500).
The image has a mode of: RGB.


In [46]:
#Function to convert image to RGB
class ConvertToRGB:
    def __call__(self, img):
        if img.mode != "RGB":
            img = img.convert("RGB")
        return img

In [48]:
#Now lets build a pipeline to convert images to RGB Resize them to a good size and convert the images to tensors
transform = transforms.Compose([
    ConvertToRGB(),
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [49]:
#Lets convert our directory into a dataset with the class labels representinting classes for doing our classification
dataset = datasets.ImageFolder(root=destination_root, transform=transform, target_transform=None)
dataset.classes

['Cassava Bacterial Blight (CBB)',
 'Cassava Brown Streak Disease (CBSD)',
 'Cassava Green Mottle (CGM)',
 'Cassava Mosaic Disease (CMD)',
 'Healthy']

In [36]:
#to investigate further how our dataset is arranged the dataset class has an attribute img which which returns path to our image and the class index
dataset.imgs[0]


('/content/train/Cassava Bacterial Blight (CBB)/1000015157.jpg', 0)

In [33]:
#we can also investigate the class indexes with the attribute class_to index which returns a dictionary of classes mapped to the index
dataset.class_to_idx

{'Cassava Bacterial Blight (CBB)': 0,
 'Cassava Brown Streak Disease (CBSD)': 1,
 'Cassava Green Mottle (CGM)': 2,
 'Cassava Mosaic Disease (CMD)': 3,
 'Healthy': 4}

In [50]:
print("Classes:")
print(dataset.classes)
print(f"That's {len(dataset.classes)} classes")
print()
print("Tensor shape for one image:")
print(dataset[0][0].shape)

Classes:
['Cassava Bacterial Blight (CBB)', 'Cassava Brown Streak Disease (CBSD)', 'Cassava Green Mottle (CGM)', 'Cassava Mosaic Disease (CMD)', 'Healthy']
That's 5 classes

Tensor shape for one image:
torch.Size([3, 224, 224])


In [51]:
#Normally we would work with the whole dataset but to make our work easier using pytorch it is reccomended we create a dataloader
batch_size = 32
dataloader = DataLoader(dataset=dataset, batch_size=batch_size)

In [55]:
#lets investigate our data loader our dataloader using iter and next
first_batch = next(iter(dataloader))
print(f"Shape of one batch: {first_batch[0].shape}")
print(f"Shape of labels: {first_batch[1].shape}")

Shape of one batch: torch.Size([32, 3, 224, 224])
Shape of labels: torch.Size([32])


## Now we have our loader to train our model but first
When we loop over this loader, it'll produce small batches of our images. This is what we want — these are the "minibatches" that will speed up our computations. In our case, each batch is  32  images, with each image  3  x  224  x  224 . It also provides us with the labels for the correct answers. This is the information we need to train a network. Normally we would split our data into training and validation test but to maximize our training accuracy pytorch works well with normalized data which is data with a mean of 0 and starndard deviation of 1, this is exactly what we are going to do, lets investigate our mean and starndard deviation then add them to our transform then we can split our training and validation sets

In [56]:
def get_mean_std(loader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in loader:
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_squared_sum += torch.mean(data**2, dim=[0, 2, 3])
        num_batches += 1

    #mean calculation
    mean = channels_sum / num_batches

    #std calculation
    std = (channels_squared_sum / num_batches - mean**2)**0.5

    return mean, std


In [57]:
mean, std = get_mean_std(dataloader)
print(f"Mean: {mean}")
print(f"Standard deviation: {std}")

Mean: tensor([0.4340, 0.4967, 0.3152])
Standard deviation: tensor([0.2183, 0.2203, 0.2071])


Now tht we have the mean and starndard deviation we can go on and normalize our data, the beast way being doing it in the transformer oipeline

In [ ]:
transform_norm = transforms.Compose(
    [
        ConvertRGB(),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ]
)